# 5 MEMORY


### ConversationBufferMemory


In [13]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)  # 메모리 생성
# return_messages=True 옵션 안 넣으면 그냥 문자열 저장 => {'history': 'Human: Hi!\nAI: How are you?'}
# 챗모델이 사용할 수 있는 형태로 저장하려면 true

memory.save_context({"input": "Hi!"}, {
                    "output": "How are you?"})  # 사용자 입력,모델 출력 저장
memory.load_memory_variables({})  # 히스토리(대화 내용 ) 확인


{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [ ]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough


model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)


def load_memory(_):
    x = memory.load_memory_variables({})
    return {"history": x["history"]}


chain = RunnablePassthrough.assign(history=load_memory) | prompt | model

inputs = {"message": "hi im bob"}
response = chain.invoke(inputs)
response


### ConversationBufferWindowMemory


In [14]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,  # 버퍼 윈도우의 사이즈 (저장할 메세지 크기)
)


def add_message(input, output):
    memory.save_context({"input": input}, {
        "output": output})


In [15]:
add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)
memory.load_memory_variables({})


{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [16]:
add_message(5, 5)
memory.load_memory_variables({})  # 1번째 것이 버려진 모습 확인 가능


{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

### ConversationSummaryMemory


In [18]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm celina, I live in South Korea", "Wow that is so cool!")


In [19]:
add_message("South Kddorea is so pretty", "I wish I could go!!!")


In [20]:
get_history()


{'history': 'Celina introduces herself as living in South Korea, which the AI finds cool. Celina mentions how pretty South Korea is, and the AI expresses a wish to visit.'}

### ConversationSummaryBufferMemory


In [21]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,  # 가능한 메세지 토큰 수의 최댓값
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm celina, I live in South Korea", "Wow that is so cool!")


In [22]:
get_history()


{'history': [HumanMessage(content="Hi I'm celina, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [23]:
add_message("South Korea is so pretty", "I wish I could go!!!")


In [24]:
get_history()


{'history': [HumanMessage(content="Hi I'm celina, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!')]}

In [32]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")


In [33]:
get_history()


{'history': [SystemMessage(content='The human introduces themselves as Celina and mentions they live in South Korea. The AI responds with enthusiasm, saying that it is cool.'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

### ConversationKGMemory


In [35]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm celina, I live in South Korea", "Wow that is so cool!")


In [36]:
memory.load_memory_variables({"input": "who is celina"})  # 묻기


{'history': [SystemMessage(content='On celina: celina lives in South Korea.')]}

In [37]:
add_message("celina likes kimchi", "Wow that is so cool!")


In [38]:
memory.load_memory_variables({"inputs": "what does celina like"})


{'history': [SystemMessage(content='On celina: celina lives in South Korea. celina likes kimchi.')]}

### Memory on LLMChain


In [41]:
from langchain.memory import ConversationSummaryBufferMemory  # 메모리
from langchain.chat_models import ChatOpenAI  # 모델
from langchain.chains import LLMChain  # LLM체인
from langchain.prompts import PromptTemplate  # 프롬프트

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",  # 템플릿 안에 메모리를 넣기
)

# 프롬프트에 지난 내화 내역(메모리에 저장된 대화/요약) 추가하기
template = """
    You are a helpful AI talking to a human.

    {chat_history} 
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,  # 체인의 프롬프트 로그들을 확인 가능
)

chain.predict(question="My name is celina ")




> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

     
    Human:My name is celina 
    You:


> Finished chain.


'Hello Celina! How can I assist you today?'

In [42]:
chain.predict(question="I live in Seoul")




> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is celina 
AI: Hello Celina! How can I assist you today? 
    Human:I live in Seoul
    You:


> Finished chain.


"That's great to hear! How can I assist you with anything related to Seoul or anything else today?"

In [43]:
chain.predict(question="What is my name?")




> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is celina 
AI: Hello Celina! How can I assist you today?
Human: I live in Seoul
AI: That's great to hear! How can I assist you with anything related to Seoul or anything else today? 
    Human:What is my name?
    You:


> Finished chain.


'Your name is Celina.'

### Chat Based Memory


In [44]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        # AI의 대답 부분을 이렇게 처리하기!
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is celina")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is celina

> Finished chain.


'Nice to meet you, Celina! How can I assist you today?'

In [45]:
chain.predict(question="I live in Seoul")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is celina
AI: Nice to meet you, Celina! How can I assist you today?
Human: I live in Seoul

> Finished chain.


'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?'

In [46]:
chain.predict(question="What is my name?")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is celina
AI: Nice to meet you, Celina! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?
Human: What is my name?

> Finished chain.


'Your name is Celina. How can I assist you further, Celina?'

### Based Memory


In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):  # 메모리 변수들을 가져오는 함수
    return memory.load_memory_variables({})["history"]  # 메모리를 로드


chain = RunnablePassthrough.assign(
    history=load_memory) | prompt | llm  # 체인 만들기


def invoke_chain(question):
    result = chain.invoke({"question": question})  # invoke
    memory.save_context(  # 사람과 AI의 메세지를 메모리에 저장
        {"input": question},
        {"output": result.content},
    )
    print(result)


In [ ]:
invoke_chain("My name is celina")


In [ ]:
invoke_chain("What is my name?")
